In [23]:
import copy, sys, pandas as pd
import random, numpy as np

sys.path.append(["../../", "../../inferring", "../../utils/DFA"])
from importlib import reload

import inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS
import utils.runLearnLib.RunLearnLib as RunLearnLib

reload(InferringDFA)
reload(RunLearnLib)
reload(utils.automats.DFA)
reload(SRS)

from inferring.InferringDFA import InferringDFA
from utils.runLearnLib.RunLearnLib import RunLearnLib
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS

In [24]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [25]:
class Record:
    def __init__(self,
                 dfa_sz,
                 lstar,
                 lstar_as, 
                 target=None):
        self.dfa_sz = dfa_sz
        self.lstar = lstar
        self.lstar_as = lstar_as
        self.target = target

    def print_record(self):
        print(
            f"|conv| = {self.dfa_sz} eq = {self.lstar[1]}, eq_with_advice = {self.lstar_as[1]}"
        )

    def get_complete_description(self):
        return self.target.print_complete_description()

    def print_target(self):
        print(self.target.print_complete_description())

In [27]:
max_number_of_states = 1000
number_of_iteration = 3
input_signs = ['a', 'b', 'c', 'd']

results = []
i = 0

helper = RunLearnLib()
# build necessary part of learnLib project
helper.compileLearnLib()

while i < number_of_iteration:
    random.seed(i)
    i += 1
    # Create random DFA with idenpotent letter and run learning process to minimize it
    dfa = DFA()
    while dfa.Q < 2:
        d = DFA()
        d.create_random_indempotent_automaton(Q=random.randint(
            max_number_of_states // 2, max_number_of_states),
                                              input_signs=input_signs,
                                              letter=input_signs[0])
        dfa, _, _ = run_learning_process(target=copy.deepcopy(d))

    f = open(
        "../../../learnlib/examples/src/main/java/de/learnlib/example/DfaEx" +
        ".txt", "w")
    f.write(d.print_complete_description())
    f.close()

    # Learn conv_d without advice
    lstar_output = helper.runLstar()
    # Learn conv_d with advice
    lstar_as_output = helper.runLstarIdemWithAdvice()

    results.append(
        Record(dfa_sz=d.Q,
               lstar=lstar_output,
               lstar_as=lstar_as_output,
               target=d))

    print(f"iter {i} done")

iter 1 done
iter 2 done
iter 3 done


In [29]:
results.sort(key=lambda x: (x.dfa_sz))

In [30]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_eq_red = df[(('Reduction', 'EQ'))].mean()
    max_eq_red = df[(('Reduction', 'EQ'))].max()
    min_eq_red = df[(('Reduction', 'EQ'))].min()
    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)

    return df

In [31]:
lstar_eqs = [r.lstar[1] for r in results]
lstar_as_eqs = [r.lstar_as[1] for r in results]

red = (
    (np.array(lstar_eqs) - np.array(lstar_as_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'DFA'), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: lstar_eqs,
    columns[2]: lstar_as_eqs,
    columns[3]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

,Target language,L*,L* with advice,Reduction
,DFA,EQ,EQ,EQ
0,568,51,36,29%
1,932,58,37,36%
2,989,56,53,5%
3,,,,
4,Max reduction,,,36%
5,Min reduction,,,5%
6,Average reduction,,,23%
